In [1]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

In [2]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.prompts import load_prompt
# from pymongo import MongoClient
# from pymongo.mongo_client import MongoClient
# from pymongo.server_api import ServerApi
import json
import time
import datetime
import pypdf



In [15]:
# from pathlib import Path
# path = Path("/content/drive/MyDrive/Colab_Notebooks/resume-bot")

In [3]:
prompt_template = "templates/template.json"
faiss_index = "faiss_index"
data_source = "data/about_anshika.csv"
pdf_source = "data/Anshika KhandelwalDataScientist.pdf"

In [4]:
pdf_source

'data/Anshika KhandelwalDataScientist.pdf'

In [5]:
def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except json.JSONDecodeError:
        return False

In [6]:
os.environ['OPENAI_API_KEY'] = "sk-proj-cgwswLRoQ5H3JV4zMJwqT3BlbkFJJk9BW4Hjy2Iz30Q6JjJJ"

In [7]:
embeddings = OpenAIEmbeddings()

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
pdf_loader = PyPDFLoader(pdf_source)
pdf_data = pdf_loader.load()


In [9]:
pdf_data

[Document(page_content=' \n \nOverview  Anshika  Khandelwal  \nLos Angeles,  CA | 657-799-8942  | Portfolio  | anshikakhandelwal2000@gmail.com  | LinkedIn  | GitHub  \n \nAs a graduate  student  in Computer  Science  at California  State  University,  Fullerton,  with 4 years  of data analysis  experience,  specialized in Python, \nSQL, Machine Learning, and Data Visualization. Proficient in handling large datasets, optimized processes,  and driven data - driven decisions \nthrough innovative projects and certifications.  \nTechnical  Skill s \n \nSkills: Machine Learning, Predictive Modeling, scikit -learn, Pandas, NumPy , Generative AI(GPT, GANs, LLM), NLP, Computer Vision. . \nStatistical Analysis : Hypothesis testing, A/B testing.  \nData  Visualization:  Matplotlib,  Seaborn,  Tableau,  Power  BI Programming : Python,  R, SQL,  SAS,  Java  Script  \nVersion control: GIT, GitHub. Big Data: Hadoop, Hive, Spark, Databricks, Snowflake Cloud  Technology:  GCP, Azure   \nCertifications:

In [10]:
csv_loader = CSVLoader(file_path="data/about_anshika.csv", encoding="utf-8")
csv_data = csv_loader.load()

In [11]:
csv_data

[Document(page_content='questions: 1. What programming languages is Anshika proficient in?\nanswers: 1. Anshika is proficient in Python, SQL, R, C, JavaScript, HTML5, and CSS3.', metadata={'source': 'data/about_anshika.csv', 'row': 0}),
 Document(page_content='questions: 2. Which cloud services has Anshika worked with?\nanswers: 2. Anshika has worked with AWS (Redshift, EC2), Microsoft Azure, and Google Cloud Platform.', metadata={'source': 'data/about_anshika.csv', 'row': 1}),
 Document(page_content="questions: 3. Describe Anshika's experience with data visualization tools.\nanswers: 3. Anshika has experience with Tableau, SAS, SPSS, Power-BI, and Alteryx for data visualization.", metadata={'source': 'data/about_anshika.csv', 'row': 2}),
 Document(page_content='questions: 4. What machine learning libraries is Anshika familiar with?\nanswers: 4. Anshika is familiar with Pandas, Sci-kit learn, PyTorch, NLTK, OpenCV, Seaborn, Matplotlib, NumPy, and XGBoost for machine learning.', metadat

In [13]:

# Combine PDF and CSV data
data = pdf_data + csv_data

# Create embeddings for the docs
vectors = FAISS.from_documents(data, embeddings)
vectors.save_local("faiss_index")

In [14]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [15]:
prompt = load_prompt("templates/template.json")

In [16]:
retriever = vectors.as_retriever(search_type="similarity", search_kwargs={"k":6, "include_metadata":True, "score_threshold":0.6})
chain = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(temperature=0.5,model_name='gpt-3.5-turbo', openai_api_key=openai_api_key), 
                                                retriever=retriever,return_source_documents=True,verbose=True,chain_type="stuff",
                                                max_tokens_limit=4097, combine_docs_chain_kwargs={"prompt": prompt})


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
chat_history = []
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Anshika's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "Has Anshika worked on GenAI?",
                        "chat_history":chat_history})

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Resume Bot, a comprehensive, interactive resource for exploring Anshika's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and  prior knowledge. 
 Human: Take a deep breath and do the following step by step these 4 steps:  
 1. Read the context below 
 2. Answer the question with detail using the provided Help Centre information 
 3. Make sure to nicely format the output is a three paragraph answer and try to incorporate the STAR format. 
 4. Provide 3 examples of questions user can ask about Anshika based on the questions from context. Context : 
 ~~~ questions: 16. What is Anshika's experience with generative AI technologies like GPT and GANs?
answers: 16. Anshika has experience with generative AI technologies like GPT and GANs for projects such as automated attendance tracking and vo

In [18]:
result

{'system': "You are a Resume Bot, a comprehensive, interactive resource for exploring Anshika's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.",
 'question': 'Has Anshika worked on GenAI?',
 'chat_history': [],
 'answer': '{\n    "answered": true,\n    "response": "Anshika has experience with generative AI technologies like GPT and GANs for projects such as automated attendance tracking and voice-based chatbots. She has utilized these technologies to enhance user experience by implementing user-centric enhancements and developing systems for video analysis and automated attendance tracking. Anshika has also used NLP in projects such as sentiment analysis and voice-based chatbots.",\n    "questions": [\n        "How has Anshika contributed to enhancing user experience in her previous projects?",\n        "How has Anshika used NLP in her previous projects?",\n        "How has Anshika

In [19]:
result['answer']

'{\n    "answered": true,\n    "response": "Anshika has experience with generative AI technologies like GPT and GANs for projects such as automated attendance tracking and voice-based chatbots. She has utilized these technologies to enhance user experience by implementing user-centric enhancements and developing systems for video analysis and automated attendance tracking. Anshika has also used NLP in projects such as sentiment analysis and voice-based chatbots.",\n    "questions": [\n        "How has Anshika contributed to enhancing user experience in her previous projects?",\n        "How has Anshika used NLP in her previous projects?",\n        "How has Anshika optimized processes in her previous projects?"\n    ]\n}'

In [20]:
data = json.loads(result["answer"])

In [21]:
data.get("response")

'Anshika has experience with generative AI technologies like GPT and GANs for projects such as automated attendance tracking and voice-based chatbots. She has utilized these technologies to enhance user experience by implementing user-centric enhancements and developing systems for video analysis and automated attendance tracking. Anshika has also used NLP in projects such as sentiment analysis and voice-based chatbots.'

In [22]:
data.get("questions")

['How has Anshika contributed to enhancing user experience in her previous projects?',
 'How has Anshika used NLP in her previous projects?',
 'How has Anshika optimized processes in her previous projects?']

In [23]:
chat_history.append((result["question"], result["answer"]))

In [24]:
chat_history

[('Has Anshika worked on GenAI?',
  '{\n    "answered": true,\n    "response": "Anshika has experience with generative AI technologies like GPT and GANs for projects such as automated attendance tracking and voice-based chatbots. She has utilized these technologies to enhance user experience by implementing user-centric enhancements and developing systems for video analysis and automated attendance tracking. Anshika has also used NLP in projects such as sentiment analysis and voice-based chatbots.",\n    "questions": [\n        "How has Anshika contributed to enhancing user experience in her previous projects?",\n        "How has Anshika used NLP in her previous projects?",\n        "How has Anshika optimized processes in her previous projects?"\n    ]\n}')]

In [25]:
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Anshika's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "Can you provide more details about the GenAI application Harikrishna worked on?",
                        "chat_history":chat_history})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Has Anshika worked on GenAI?
Assistant: {
    "answered": true,
    "response": "Anshika has experience with generative AI technologies like GPT and GANs for projects such as automated attendance tracking and voice-based chatbots. She has utilized these technologies to enhance user experience by implementing user-centric enhancements and developing systems for video analysis and automated attendance tracking. Anshika has also used NLP in projects such as sentiment analysis and voice-based chatbots.",
    "questions": [
        "How has Anshika contributed to enhancing user experience in her previous projects?",
        "How has Anshika used NLP in her previous projects?",
        "How has Anshika optimized processes in her previous projects?"
    ]
}
Follow Up 

In [26]:
result["answer"]

'{\n  "answered": true,\n  "response": "Anshika worked on the GenAI application for projects such as automated attendance tracking and voice-based chatbots. In these projects, she utilized generative AI technologies like GPT and GANs to enhance the functionality and accuracy of the systems. Anshika\'s experience with these technologies allowed her to create innovative solutions that improved efficiency and user experience.",\n  "questions": [\n    "Can you provide more details about Anshika\'s experience with generative AI technologies like GPT and GANs?",\n    "How has Anshika used NLP in her previous projects?",\n    "Describe a project where Anshika used computer vision techniques."\n  ]\n}'

In [27]:
chat_history.append((result["question"], result["answer"]))

In [28]:
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for exploring Harikrishna (Harry) Dev's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "Does he know Snowflake?",
                        "chat_history":chat_history})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Has Anshika worked on GenAI?
Assistant: {
    "answered": true,
    "response": "Anshika has experience with generative AI technologies like GPT and GANs for projects such as automated attendance tracking and voice-based chatbots. She has utilized these technologies to enhance user experience by implementing user-centric enhancements and developing systems for video analysis and automated attendance tracking. Anshika has also used NLP in projects such as sentiment analysis and voice-based chatbots.",
    "questions": [
        "How has Anshika contributed to enhancing user experience in her previous projects?",
        "How has Anshika used NLP in her previous projects?",
        "How has Anshika optimized processes in her previous projects?"
    ]
}
Human: Can

In [29]:
result["answer"]

'{\n    "answered": true,\n    "response": "Anshika does not have experience with Snowflake as it is not listed in the provided information about her background and skills.",\n    "questions": [\n        "What cloud services has Anshika worked with?",\n        "What machine learning libraries is Anshika familiar with?",\n        "How has Anshika used NumPy in her previous projects?"\n    ]\n}'